In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
import numpy as np
from reproject import reproject_interp
import os 
from tqdm import tqdm 
from astropy.table import join

plt.style.use('paper.mplstyle')

from imports import *

/Users/abarnes/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

# table = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue.fits' %root_dir)
# table_wprops = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_wassociations_wprops.fits' %root_dir)
# table_wasso = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_wassociations.fits' %root_dir)

table = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_fixediso_10pc.fits' %root_dir)
table_wprops = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_wassociations_wprops_fixediso_10pc.fits' %root_dir)
table_wasso = QTable.read('%s/data_hstha_nebulae_catalogue/all/nebulae_catalogue_wassociations_fixediso_10pc.fits' %root_dir)

print(f'Number of objects in table pre-masking (for stars etc): {len(table)}')

table_muse = QTable.read('%s/data_misc/Nebulae_catalogue_v4/Nebulae_catalogue_v4.fits' %root_dir)

print(f'Number of stars in HST table: {np.sum(table["flag_star_MUSE"])}')
print(f'Number of MUSE edge in HST table: {np.sum(table["flag_edge_MUSE"])}')
print(f'Number of HST edge in HST table: {np.sum(table["flag_edge_hst"])}')

print(f'Number of stars in MUSE table: {np.sum(table_muse["flag_star"])}')
print(f'Number of MUSE edge in MUSE table: {np.sum(table_muse["flag_edge"])}')

table = table[(table['flag_star_MUSE']!=1) & (table['flag_edge_MUSE']!=1) & (table["flag_edge_hst"]!=1)] # remove stars and edge objects
table_wprops = table_wprops[(table_wprops['flag_star_MUSE']!=1) & (table_wprops['flag_edge_MUSE']!=1) & (table_wprops["flag_edge_hst"]!=1)] # remove stars and edge objects
table_wasso = table_wasso[(table_wasso['flag_star_MUSE']!=1) & (table_wasso['flag_edge_MUSE']!=1) & (table_wasso["flag_edge_hst"]!=1)] # remove stars and edge objects
table_muse = table_muse[(table_muse['flag_star']!=1) & (table_muse['flag_edge']!=1)] # remove stars and edge objects

mask_hii = table_wasso['HII_class_v3_MUSE']==1
mask_noasso = ~table_wasso['no_associations_association_nuv_32pc']
table_wasso = table_wasso[mask_noasso&mask_hii]

if len(table) != len(table_wprops):
    print('Tables ARE NOT the same length! - PLEASE CHECK')

print(f'Number of objects in table post-masking (for stars etc): {len(table)}')

Number of objects in table pre-masking (for stars etc): 5781
Number of stars in HST table: 57.0
Number of MUSE edge in HST table: 64.0
Number of HST edge in HST table: 108
Number of stars in MUSE table: 98.0
Number of MUSE edge in MUSE table: 611.0
Number of objects in table post-masking (for stars etc): 5559


In [3]:
def get_files(endswith, root_dir): 

    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(endswith):
                if '628e' in file: 
                    continue
                if '628mosaic' in file: 
                    continue
                if 'f550m' in root:
                    continue
                if 'f547m' in root:
                    continue
                else: 
                    # print(os.path.join(root, file))
                    file_list += [os.path.join(root, file)]

    print('TOTAL NUMBER OF FILES:', len(file_list))

    return file_list

def match_files(files1, files2, root_dir, id=0, verbose=True):
    matched_files = []
    for file1 in files1:
        for file2 in files2:
            if file1.split(root_dir)[-1].split('/')[2] == file2.split(root_dir)[-1].split('/')[2]:
                if verbose: 
                    print(file1, file2)
                matched_files.append(file2)
                break

    if len(files1) != len(matched_files):
        print('WARNING: DIFFERENT NUMBER OF FILES')

    return matched_files


files_maps = get_files('_hst_ha.fits', root_dir)
files_nebs_ = get_files('_nebmask.fits', root_dir)
files_nebs = match_files(files_maps, files_nebs_, root_dir, 2)
print('TOTAL NUMBER OF FILES:', len(files_nebs))

TOTAL NUMBER OF FILES: 19
TOTAL NUMBER OF FILES: 20
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/ngc3627_hst_ha.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/muse/NGC3627_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/ngc4254_hst_ha.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/muse/NGC4254_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/ngc1512_hst_ha.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/muse/NGC1512_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/ngc1433_hst_ha.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/muse/NGC1433_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/ngc5068_hst_ha.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/dat

In [ ]:
# Regrid all MUSE NebCat data to HST pixels

regrid_musecat = False
if regrid_musecat: 

    nfiles = len(files_maps)
    for i in tqdm(range(nfiles)): 

        if 'ngc2835' not in files_maps[i]:
            continue

        file_maps, file_nebs = files_maps[i], files_nebs[i]

        # Loading files...
        hstha_hdu = fits.open(file_maps)[0]
        muscat_hdu = fits.open(file_nebs)[0]

        # Update arrays
        muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
        muscat_hdu.data[muscat_hdu.data==-1] = np.nan

        # Interpolate masks
        muscat_data_re, _ = reproject_interp(muscat_hdu, hstha_hdu.header, order='nearest-neighbor')
        muscat_data_re[np.isnan(hstha_hdu.data)] = np.nan

        muscat_hdu_re = fits.PrimaryHDU(muscat_data_re, hstha_hdu.header)

        output_file = file_nebs.replace('.fits', '_regrid.fits')
        print('Writing file: %s' %output_file)
        
        muscat_hdu_re.writeto(output_file, overwrite=True)
        # muscat_hdu.writeto(file_maps.replace('_hst_ha.fits', '_muse_nebmask.fits'), overwrite=True)

files_nebs_r = get_files('nebmask_regrid.fits', root_dir)

100%|██████████| 19/19 [00:19<00:00,  1.03s/it]

Writing file: /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc2835/muse/NGC2835_nebmask_regrid.fits
TOTAL NUMBER OF FILES: 19


In [10]:
data_table = []
nfiles = len(files_maps)
table_muse_masked = []

for i in range(nfiles): 

    file_maps = files_maps[i]
    file_nebs = files_nebs[i]
    file_nebs_r = files_nebs_r[i]

    gal_name = file_nebs_r.split(root_dir)[-1].split('/')[2]
    gal_name = gal_name.upper()
    gal_name = gal_name.replace('1365N', '1365')
    gal_name = gal_name.replace('2835S', '2835')
    gal_name = gal_name.replace('628C', '0628')

    mask_gal_hst = table['gal_name'] == gal_name
    mask_gal_hst_asso = table_wasso['gal_name'] == gal_name
    mask_gal_muse = table_muse['gal_name'] == gal_name

    muscat_hdu = fits.open(file_nebs)[0]
    muscat_hdu_r = fits.open(file_nebs_r)[0]

    # ALL regions from MUSE map within HST coverage - inc stars
    map_ids = np.unique(muscat_hdu_r.data)
    map_ids = map_ids[~np.isnan(map_ids)]

    # ALL regions from MUSE map with MUSE coverage - inc stars
    map_ids_noregrid = np.unique(muscat_hdu.data)
    map_ids_noregrid = map_ids_noregrid[~np.isnan(map_ids_noregrid)]
    map_ids_noregrid = map_ids_noregrid[map_ids_noregrid!=-1]

    # ALL regions in MUSE table within MUSE coverage
    tab_ids = table_muse[mask_gal_muse]['region_ID'] 

    # Create mask where tab_ids are in map_ids
    mask = np.zeros(len(tab_ids), dtype=bool)
    for i, tab_id in enumerate(tab_ids): 
        mask[i] = tab_id in map_ids

    # Get masked tables 
    table_muse_masked_gal = table_muse[mask_gal_muse][mask]     # MUSE table within HST coverage per galaxy
    table_muse_masked += [table_muse_masked_gal]                # MUSE table within HST coverage all
    table_hst_masked = table[mask_gal_hst]
    table_wprops_masked = table_wprops[mask_gal_hst]
    table_wasso_masked = table_wasso[mask_gal_hst_asso]

    nreg_muse_map = len(map_ids_noregrid)                      # total number of regions in MUSE map - inc stars
    nreg_muse = len(table_muse[mask_gal_muse])                 # total number of regions in MUSE table
    nreg_muse_masked = len(table_muse[mask_gal_muse][mask])    # total number of regions in MUSE within HST coverage
    nreg_hst = len(table_hst_masked)                           # total number of regions in HST (within MUSE coverage)

    nreg_hst_complex_0 = sum(table_hst_masked['complexity_score'] <= 1) #simple
    nreg_hst_complex_1 = sum((table_hst_masked['complexity_score'] > 1) & (table_hst_masked['complexity_score'] <= 5)) #intermediate
    nreg_hst_complex_2 = sum(table_hst_masked['complexity_score'] > 5) #complex

    nreg_hst_hii = len(table_hst_masked[table_hst_masked['HII_class_v3_MUSE']==1]) # total number of HII regions in HST (within MUSE coverage)
    nreg_hst_ne = len(table_wprops_masked[(table_wprops_masked['HII_class_v3_MUSE']==1) & ~np.isnan(table_wprops_masked['ne'])]) # total number of HII regions with ne in HST (within MUSE coverage)

    nreg_hst_asso = len(table_wasso_masked)

    mask_center_hst = table_hst_masked['Environment_MUSE'] != 1
    mask_center_mus = table_muse_masked_gal['Environment'] != 1

    lum_frac = table_hst_masked['HA6562_LUMINOSITY_HST']/table_hst_masked['HA6562_LUMINOSITY_MUSE']
    lum_frac_mask = lum_frac<2
    lum_frac = lum_frac[lum_frac_mask]

    lum_tot_muse = np.nansum(table_muse_masked_gal['Lum_HA6562_CORR'])           # Total MUSE luminosity of all regions in MUSE within HST coverage
    lum_tot_hst_muse = np.nansum(table_hst_masked['HA6562_LUMINOSITY_MUSE'][lum_frac_mask])     # Total MUSE luminosity of all regions in HST within MUSE coverage
    lum_tot_hst = np.nansum(table_hst_masked['HA6562_LUMINOSITY_HST'][lum_frac_mask])           # Total HST luminosity of all regions in HST within MUSE coverage

    lum_tot_muse_cent = np.nansum(table_muse_masked_gal['Lum_HA6562_CORR'][mask_center_mus])           # Total MUSE luminosity of all regions in MUSE within HST coverage
    lum_tot_hst_muse_cent = np.nansum(table_hst_masked['HA6562_LUMINOSITY_MUSE'][mask_center_hst])     # Total MUSE luminosity of all regions in HST within MUSE coverage
    lum_tot_hst_cent = np.nansum(table_hst_masked['HA6562_LUMINOSITY_HST'][mask_center_hst])           # Total HST luminosity of all regions in HST within MUSE coverage

    nfrac = (nreg_hst/nreg_muse_masked)*100
    lfrac1 = (lum_tot_hst.value/lum_tot_muse.value)*100
    lfrac2 = np.nanmean(np.array(lum_frac.value))*100

    gal_name = gal_name.replace('0628', '628')
    gal_name = gal_name.replace('NGC', 'NGC~')
    gal_name = gal_name.replace('IC', 'IC~')

    gal_data = [gal_name]
    gal_data += [nreg_muse, 
                nreg_muse_masked, 
                nreg_hst,
                nfrac,
                nreg_hst_complex_0,
                nreg_hst_complex_1,
                nreg_hst_complex_2,
                nreg_hst_hii,
                nreg_hst_ne,
                nreg_hst_asso,
                ]
    
    print(gal_data)

    data_table.append(gal_data)

# MUSE table within HST coverage - all galaxies
table_muse_masked = QTable(np.concatenate(table_muse_masked))
table_muse_masked.sort(['gal_name', 'region_ID'])
table_muse_masked.write('%s/data_misc/Nebulae_catalogue_v3/Nebulae_catalogue_v3_HSTFOV.fits' %root_dir, overwrite=True)

['NGC~3627', 1598, 1122, 294, 26.203208556149733, 200, 55, 39, 274, 160, 200]
['NGC~4254', 2940, 2464, 766, 31.087662337662337, 485, 213, 68, 746, 396, 532]
['NGC~1512', 623, 489, 92, 18.813905930470348, 68, 11, 13, 85, 28, 55]
['NGC~1433', 1717, 1018, 173, 16.994106090373283, 122, 35, 16, 137, 34, 71]
['NGC~5068', 1781, 1699, 126, 7.416127133608005, 105, 17, 4, 124, 98, 104]
['NGC~4321', 1806, 1265, 369, 29.169960474308297, 254, 70, 45, 347, 150, 248]
['NGC~1365', 1409, 888, 196, 22.07207207207207, 108, 38, 50, 144, 74, 86]
['NGC~4303', 3029, 2650, 757, 28.566037735849058, 490, 183, 84, 718, 393, 483]
['NGC~2835', 1068, 1007, 264, 26.216484607745777, 191, 60, 13, 243, 161, 178]
['NGC~1087', 1004, 995, 305, 30.65326633165829, 215, 63, 27, 302, 128, 177]
['NGC~7496', 758, 728, 138, 18.956043956043956, 85, 34, 19, 132, 51, 82]
['NGC~1385', 1023, 1023, 326, 31.867057673509287, 191, 69, 66, 322, 196, 194]
['IC~5332', 786, 756, 42, 5.555555555555555, 41, 1, 0, 41, 12, 30]
['NGC~3351', 1247,

In [11]:
len_table_hst = len(table)
len_table_muse = len(table_muse)
len_table_hst_wprops = len(table_wprops)
len_table_muse_masked = len(table_muse_masked)
len_table_muse_masked_hii = len(table_muse_masked[table_muse_masked['HII_class_v3']==1])
len_table_hst_hii = len(table[table['HII_class_v3_MUSE']==1])
len_table_hst_hii_ne = len(table_wprops[(table_wprops['HII_class_v3_MUSE']==1) & ~np.isnan(table_wprops['ne'])])
len_table_hst_asso = len(table_wasso)

len_hst_complex_0 = sum(table_wprops['complexity_score'] <= 1) #simple
len_hst_complex_1 = sum((table_wprops['complexity_score'] > 1) & (table_wprops['complexity_score'] <= 5)) #intermediate
len_hst_complex_2 = sum(table_wprops['complexity_score'] > 5) #complex

print('Number of regions in MUSE table (all 19 galaxies):', len_table_muse)
print('Number of regions in MUSE table within HST coverage:', len_table_muse_masked)
print('Number of HII regions in MUSE table within HST coverage:', len_table_muse_masked_hii)

print('Number of regions in HST table:', len_table_hst)
print('Number of regions in HST table with properties:', len_table_hst_wprops)

print('Number of simple regions in HST table:', len_hst_complex_0)
print('Number of intermediate regions in HST table:', len_hst_complex_1)
print('Number of complex regions in HST table:', len_hst_complex_2)

print('Number of HII regions in HST table:', len_table_hst_hii)
print('Number of HII regions in HST table with ne:', len_table_hst_hii_ne)

print('Number of associations in HST table:', len_table_hst_asso)

Number of regions in MUSE table (all 19 galaxies): 30790
Number of regions in MUSE table within HST coverage: 25910
Number of HII regions in MUSE table within HST coverage: 19528
Number of regions in HST table: 5559
Number of regions in HST table with properties: 5559
Number of simple regions in HST table: 3728
Number of intermediate regions in HST table: 1201
Number of complex regions in HST table: 630
Number of HII regions in HST table: 5198
Number of HII regions in HST table with ne: 2606
Number of associations in HST table: 3484


In [12]:
# Create a new QTable 
new_columns = ['gal_name', 'nreg_muse', 'nreg_muse_masked', 'nreg_hst', 'frac_nreg',
               'nreg_hst_complex_0', 'nreg_hst_complex_1', 'nreg_hst_complex_2',
               'nreg_hst_hii', 'nreg_hst_hii_ne', 'nreg_hst_asso',
               ]
new_table = QTable(rows=data_table, names=new_columns)

new_table.sort('gal_name')

nreg_tot_muse = np.sum(new_table['nreg_muse'])
nreg_tot_musemasked = np.sum(new_table['nreg_muse_masked'])
nreg_tot_hst = np.sum(new_table['nreg_hst'])
nreg_tot_hst_complex_0 = np.sum(new_table['nreg_hst_complex_0'])
nreg_tot_hst_complex_1 = np.sum(new_table['nreg_hst_complex_1'])
nreg_tot_hst_complex_2 = np.sum(new_table['nreg_hst_complex_2'])
nreg_tot_hst_asso = np.sum(new_table['nreg_hst_asso'])
nreg_tot_hst_hii = np.sum(new_table['nreg_hst_hii'])
nreg_tot_hst_ne = np.sum(new_table['nreg_hst_hii_ne'])

lum_tot_muse = np.nansum(table_muse_masked['Lum_HA6562_CORR'])    # Total MUSE luminosity of all regions in MUSE within HST coverage
lum_tot_hst_muse = np.nansum(table['HA6562_LUMINOSITY_MUSE'])     # Total MUSE luminosity of all regions in HST within MUSE coverage
lum_tot_hst = np.nansum(table['HA6562_LUMINOSITY_HST'])           # Total HST luminosity of all regions in HST within MUSE coverage

mask_center_hst = table['Environment_MUSE'] != 1
mask_center_mus = table_muse_masked['Environment'] != 1

lum_tot_muse_cent = np.nansum(table_muse_masked['Lum_HA6562_CORR'][mask_center_mus])           # Total MUSE luminosity of all regions in MUSE within HST coverage
lum_tot_hst_muse_cent = np.nansum(table['HA6562_LUMINOSITY_MUSE'][mask_center_hst])     # Total MUSE luminosity of all regions in HST within MUSE coverage
lum_tot_hst_cent = np.nansum(table['HA6562_LUMINOSITY_HST'][mask_center_hst])          # Total HST luminosity of all regions in HST within MUSE coverage

nfrac = (nreg_tot_hst/nreg_tot_musemasked)*100
# lfrac = (lum_tot_hst/lum_tot_muse).value*100

# sum columns in new_table and append at end with row "total"
new_table.add_row(['Total', 
                    nreg_tot_muse, 
                    nreg_tot_musemasked, 
                    nreg_tot_hst, 
                    nfrac,
                    nreg_tot_hst_complex_0,
                    nreg_tot_hst_complex_1,
                    nreg_tot_hst_complex_2,
                    nreg_tot_hst_hii,
                    nreg_tot_hst_ne,
                    nreg_tot_hst_asso,
                ])

formats = {'frac_nreg': '%0.1f'}
new_table.write('./tabs/galmap_compprops.tex', formats=formats, overwrite=True)
new_table

gal_name,nreg_muse,nreg_muse_masked,nreg_hst,frac_nreg,nreg_hst_complex_0,nreg_hst_complex_1,nreg_hst_complex_2,nreg_hst_hii,nreg_hst_hii_ne,nreg_hst_asso
str8,int64,int64,int64,float64,int64,int64,int64,int64,int64,int64
IC~5332,786,756,42,5.555555555555555,41,1,0,41,12,30
NGC~1087,1004,995,305,30.65326633165829,215,63,27,302,128,177
NGC~1300,1448,1437,245,17.049408489909535,170,60,15,227,84,113
NGC~1365,1409,888,196,22.07207207207207,108,38,50,144,74,86
NGC~1385,1023,1023,326,31.867057673509287,191,69,66,322,196,194
NGC~1433,1717,1018,173,16.994106090373283,122,35,16,137,34,71
NGC~1512,623,489,92,18.813905930470348,68,11,13,85,28,55
NGC~1566,2330,2049,497,24.255734504636408,314,110,73,459,243,345
NGC~1672,1559,1535,427,27.81758957654723,262,89,76,389,207,256
